# Imports

In [1]:
import pybullet as p
import random
import numpy as np
from sim_class import Simulation
import imageio

# Test simulation

In the code below I will manually run the code with different value to check behavior of the simulation

In [2]:
# Initialize the simulation with a specified number of agents
sim = Simulation(num_agents=1)  # For one robot

# Example action: Move joints with specific velocities
velocity_x = 0.0
velocity_y = 0.0     
velocity_z = 0.1
drop_command = 0
actions = [[velocity_x, velocity_y, velocity_z, drop_command]]

# Run the simulation for a specified number of steps
state = sim.run(actions, num_steps=100)
print(state)
sim.close()

{'robotId_1': {'joint_states': {'joint_0': {'position': 4.733309137304465e-06, 'velocity': 3.40136667881805e-21, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': 0.0007842998760603653}, 'joint_1': {'position': -2.009053568053385e-05, 'velocity': -5.921858744589238e-05, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': -0.0003005933208470764}, 'joint_2': {'position': 0.07371562508408833, 'velocity': 0.09997532001169776, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': 122.69211031438819}}, 'robot_position': [-3.6841702859100335e-06, 2.014274755314002e-05, 0.10371563302059741], 'pipette_position': [0.073, 0.0895, 0.1932]}}


In [3]:
sim = Simulation(num_agents=1)
for i in range(300):
        velocity_x = random.uniform(0, 0.5)
        velocity_y = random.uniform(0, 0.5)
        velocity_z = random.uniform(0, 0.5)
        drop_command = random.randint(0, 1)

        actions = [[velocity_x, velocity_y, velocity_z, drop_command],
                [velocity_x, velocity_y, velocity_z, drop_command]]

        state = sim.run(actions)
        print(state)
sim.close()

{'robotId_1': {'joint_states': {'joint_0': {'position': -0.00045802218904649454, 'velocity': -0.10992532537115869, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': -500.00000000000006}, 'joint_1': {'position': -0.0005576807802752383, 'velocity': -0.13384338726605718, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': -500.00000000000006}, 'joint_2': {'position': 0.00032802946457974905, 'velocity': 0.07872707149913977, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': 354.3553528985812}}, 'robot_position': [0.00045802218904649454, 0.0005576854257068343, 0.030328027879529273], 'pipette_position': [0.0735, 0.0901, 0.1198]}}
{'robotId_1': {'joint_states': {'joint_0': {'position': -0.0013739762001764954, 'velocity': -0.21982896267120022, 'reaction_forces': (0.0, 0.0, 0.0, 0.0, 0.0, 0.0), 'motor_torque': -500.00000000000006}, 'joint_1': {'position': -0.0016729520155574153, 'velocity': -0.2676650964677225, 'reaction_forces': (0.0, 0.0, 0.0, 0

Summary:

- `x/-x` == getting closer/further
- `y/-y` == right/left
- `z/-z` == up/down (up does jump right after start simulation - unknown reason)
- the position coordinates are not stable when reached the edge of work envelope. During work envelope checking tolerance or treshold has to be applied to avoid infinite loop
- `x/y` velocities are applied as negative of action, while `Z` is applied directly (no inversion)

In [53]:
def drive_until_still(sim, axis, vel, tol=0.00001, stable_count=5):
    """Drive the pipette along a specified axis until it becomes stable.
    Args:
        sim: Simulation object
        axis: "x", "y", or "z"
        vel: velocity to drive along the axis (positive or negative)
        tol: tolerance for stability
        stable_count: number of consecutive stable readings required
    Returns:
        pip: final pipette position
        """
    idx = {"x": 0, "y": 1, "z": 2}[axis]
    history = [] # for checking stability

    while True:
        action = [0.0, 0.0, 0.0, 0]
        action[idx] = vel
        state = sim.run([action], num_steps=1)
        pip = next(iter(state.values()))["pipette_position"]

        # check for stability
        history.append(pip[idx])
        if len(history) > stable_count:
            history.pop(0)
            if max(history) - min(history) < tol:
                return pip

# initialize simulation
sim = Simulation(num_agents=1)

# z bounds
z_top = drive_until_still(sim, "z", +0.4)
z_bottom = drive_until_still(sim, "z", -0.4)

# x bounds
x_pos = drive_until_still(sim, "x", +0.4)
x_neg = drive_until_still(sim, "x", -0.4)

# y bounds
y_pos = drive_until_still(sim, "y", +0.4)
y_neg = drive_until_still(sim, "y", -0.4)

# save gif where duration is time in seconds per frame
# imageio.mimwrite("work_envelope.gif", gif_frames, format="GIF", duration=1/60)

# display results
# (x, y, z) positions
print("Z top:", z_top[2])
print("Z bottom:", z_bottom[2])
print("X +limit:", x_pos[0])
print("X -limit:", x_neg[0])
print("Y +limit:", y_pos[1])
print("Y -limit:", y_neg[1])

sim.close()

Z top: 0.2896
Z bottom: 0.1694
X +limit: 0.2531
X -limit: -0.1871
Y +limit: 0.2195
Y -limit: -0.1706


In [128]:
def move_towards(sim, axis, vel, tol=0.0001, stable_count=5):
    """Drive the pipette along a specified axis until it becomes stable.
    Args:
        sim: Simulation object
        axis: "x", "y", or "z"
        vel: velocity to drive along the axis (positive or negative)
        tol: tolerance for stability
        stable_count: number of consecutive stable readings required
    Returns:
        pip: final pipette position
        """
    idx = {"x": 0, "y": 1, "z": 2}[axis]
    history = [] # for checking stability
    frames = [] # for gif

    while True:
        action = [0.0, 0.0, 0.0, 0]
        action[idx] = vel
        state = sim.run([action], num_steps=1)
        pip = next(iter(state.values()))["pipette_position"]

        rgba = np.array(sim.current_frame, dtype=np.uint8)
        rgba = rgba.reshape((240, 320, 4)) # hardcoded in sim_class.py
        frames.append(rgba)
        
        history.append(pip[idx])
        if len(history) > stable_count:
            history.pop(0)
            if max(history) - min(history) < tol:
                return pip, frames


In [129]:
# Path planning corners
bottom_xy = [(+1, -1), (-1, -1), (-1, +1), (+1, +1)]
top_xy    = [(-1, -1), (-1, +1), (+1, +1), (+1, -1)]

[imageio.mimwrite](https://imageio.readthedocs.io/en/v2.9.0/userapi.html#imageio.mimwrite) is used to save the `work_envelope.gif` (requirement)

In [130]:
sim = Simulation(num_agents=1, rgb_array=True)

frames_all = []

# bottom Z
_, frames = move_towards(sim, "z", 0.6)
frames_all.extend(frames)

for x, y in top_xy:
    _, frames = move_towards(sim, "x", 2 * x)
    frames_all.extend(frames)

    _, frames = move_towards(sim, "y", 2 * y)
    frames_all.extend(frames)

_, frames = move_towards(sim, "z", -0.6)
frames_all.extend(frames)

for x, y in bottom_xy:
    _, frames = move_towards(sim, "x", 2 * x)
    frames_all.extend(frames)

    _, frames = move_towards(sim, "y", 2 * y)
    frames_all.extend(frames)

imageio.mimwrite("work_envelope.gif", frames_all, duration=1/30)

sim.close()